# Next steps 
- connect precip to DF,
- add new sites (e.g., regionval) to training DF with all the respective spatial resolution information
- connect regional data together to train model
- connect different regions
- add precipitation phase features (seasonal accumulated rain precip, seasonal accumulated snow precip as a function of temperature)
- explore adding other features stemming from SNOTEL, remote sensing (LULC), Snow Classifications (Sturms), energy balance

Put all units in SI, while it should not matter for model training since they are being normalized, they will be more interpretable.

In [ ]:
import os
import pandas as pd

HOME = os.path.expanduser('~')

region = 'S_Sierras'

#ASO observations
aso_swe_files_folder_path = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/GeoObsDFs"

obs1 = pd.read_csv(f"{aso_swe_files_folder_path}/GeoObsdfs_20130429.parquet")

obs1.head()

In [5]:
Precippath = f"{HOME}/SWEMLv2.0/data/Precipitation/{region}/{output_res}M_NLDAS_Precip"
ppt = pd.read_hdf(f"{Precippath}/WY{year}.h5", key = '11N_cell_-119.59073383567106_38.18624284828164')
ppt.head()

Loading goeospatial meta data for grids in S_Sierras
Loading all available processed ASO observations for the S_Sierras at 100M resolution


100%|██████████| 99/99 [00:00<00:00, 1269835.16it/s]


In [7]:
aso_gdf

,cell_id,cen_lon,cen_lat,geometry,Elevation_m,Slope_Deg,Aspect_Deg
0,11N_cell_-119.59073383567106_38.18624284828164,-119.590255,38.185854,POINT (-119.5902546710551 38.18585423359679),3167,0,180
1,11N_cell_-119.58959364631137_38.186209698720205,-119.589255,38.185854,POINT (-119.58925467105512 38.18585423359679),3168,2,18
2,11N_cell_-119.59309813700962_38.184509449472536,-119.593255,38.184854,POINT (-119.5932546710551 38.1848542335968),3156,5,342
3,11N_cell_-119.59195797185825_38.18447632413384,-119.592255,38.184854,POINT (-119.59225467105512 38.1848542335968),3158,5,34
4,11N_cell_-119.59081780857791_38.184443187748315,-119.591255,38.184854,POINT (-119.59125467105513 38.1848542335968),3165,8,32
...,...,...,...,...,...,...,...
1361693,11N_cell_-118.38164158972388_36.55407079223968,-118.381813,36.553843,POINT (-118.38181258102036 36.55384258766223),3107,35,57
1361694,11N_cell_-118.380527494837_36.554027541819025,-118.380813,36.553843,POINT (-118.38081258102038 36.55384258766223),3122,32,63
1361695,11N_cell_-118.35819223061885_36.554059069937786,-118.357813,36.553843,POINT (-118.35781258102035 36.55384258766223),3165,16,122
1361696,11N_cell_-118.3570781729364_36.55401560043055,-118.356813,36.553843,POINT (-118.35681258102038 36.55384258766223),3162,17,90


In [1]:

import os
import pandas as pd

HOME = os.path.expanduser('~')

region = 'S_Sierras'

#ASO observations
aso_swe_files_folder_path = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/Obsdf"

obs1 = pd.read_csv(f"{aso_swe_files_folder_path}/20130403_ObsDF.parquet")

obs1

,Unnamed: 0,cell_id,Date,swe,nearest_site_1,nearest_site_4,nearest_site_2,nearest_site_6,nearest_site_3,nearest_site_5
0,0,11N_cell_-119.59073383567106_38.18624284828164,2013-04-03,0.972643,44.52,0.1,49.00,18.800000,0.00,18.9
1,1,11N_cell_-119.58959364631137_38.186209698720205,2013-04-03,0.536205,44.52,0.1,49.00,18.800000,0.00,18.9
2,2,11N_cell_-119.59309813700962_38.184509449472536,2013-04-03,0.719424,44.52,0.1,49.00,18.800000,0.00,18.9
3,3,11N_cell_-119.59195797185825_38.18447632413384,2013-04-03,0.618731,44.52,0.1,49.00,18.800000,0.00,18.9
4,4,11N_cell_-119.59081780857791_38.184443187748315,2013-04-03,0.471944,44.52,0.1,49.00,18.800000,0.00,18.9
...,...,...,...,...,...,...,...,...,...,...
120428,120428,11N_cell_-119.27205242391986_37.7395618848736,2013-04-03,0.339904,18.79,20.1,17.65,13.596211,8.44,13.0
120429,120429,11N_cell_-119.27091951465975_37.73952599896746,2013-04-03,0.921304,18.79,20.1,17.65,13.596211,8.44,13.0
120430,120430,11N_cell_-119.26978660741051_37.739490102201515,2013-04-03,1.935530,18.79,20.1,17.65,13.596211,8.44,13.0
120431,120431,11N_cell_-119.26978660741051_37.739490102201515,2013-04-03,1.843666,18.79,20.1,17.65,13.596211,8.44,13.0


In [ ]:
cols = [
    'cell_id', 'Date',  'cen_lat', 'cen_lon', 'geometry', 'Elevation_m', 'Slope_Deg',
    'Aspect_Deg', 'swe', 'nearest_site_1', 'nearest_site_2', 'nearest_site_3', 'nearest_site_4', 
    'nearest_site_5', 'nearest_site_6'
    ]